<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
import skillsnetwork 


In [2]:
await skillsnetwork.prepare("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip", overwrite=True)

  0%|          | 0/80045 [00:00<?, ?it/s]

Saved to '.'


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
import tensorflow as tf
tf.config.threading.set_intra_op_parallelism_threads(8)
tf.config.threading.set_inter_op_parallelism_threads(8)

2024-06-16 17:27:34.451686: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 17:27:35.421659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
batch_size = 50

In [5]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')


Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [6]:
num_classes = 2

vgg16 = keras.models.Sequential()
vgg16.add(VGG16(include_top=False, pooling='avg', weights='imagenet'))
vgg16.add(Dense(num_classes, activation='softmax'))
vgg16.layers[0].trainable = False


2024-06-16 17:27:39.033177: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-16 17:27:39.067447: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
steps_per_epoch_training = train_generator.n // train_generator.batch_size
steps_per_epoch_validation = validation_generator.n // validation_generator.batch_size
vgg16.summary()
num_epochs = 1

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ ?                      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [8]:
fit_history = vgg16.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

/home/mj/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-06-16 17:27:41.560641: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 642252800 exceeds 10% of free system memory.
2024-06-16 17:27:41.884835: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 642252800 exceeds 10% of free system memory.
2024-06-16 17:27:43.020308: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 321126400 exceeds 10% of free system memory.
2024-06-16 17:27:43.430095: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 321126400 exceeds 10% of free system memory.


  1/600 ━━━━━━━━━━━━━━━━━━━━ 1:31:36 9s/step - accuracy: 0.3000 - loss: 1.2711

2024-06-16 17:27:48.975611: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 642252800 exceeds 10% of free system memory.


600/600 ━━━━━━━━━━━━━━━━━━━━ 5941s 10s/step - accuracy: 0.8829 - loss: 0.2553 - val_accuracy: 0.9961 - val_loss: 0.0185


In [9]:
vgg16.save('classifier_vgg16_model.keras')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [10]:
# model file = classifier_resnet_model.keras

resnet = keras.models.load_model('classifier_resnet_model.keras')

In [11]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical')


Found 500 images belonging to 2 classes.


In [12]:
eval_vgg16 = vgg16.evaluate(test_generator)
print("VGG16 Model Accuracy on Test Set: ", eval_vgg16[1])

10/10 ━━━━━━━━━━━━━━━━━━━━ 75s 8s/step - accuracy: 0.9984 - loss: 0.0168
VGG16 Model Accuracy on Test Set:  0.9959999918937683


In [13]:
eval_resnet = resnet.evaluate(test_generator)
print("ResNet model accuracy on test data: ", eval_resnet[1])

10/10 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step - accuracy: 0.9989 - loss: 0.0106
ResNet model accuracy on test data:  0.9980000257492065


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [14]:
predictions_vgg16 = vgg16.predict(test_generator)

10/10 ━━━━━━━━━━━━━━━━━━━━ 75s 7s/step


In [15]:
predictions_resnet = resnet.predict(test_generator)

10/10 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step


In [26]:
import random

print("VGG16 Predictions: ")
indices = random.sample(range(len(predictions_vgg16)), 5)
for i in indices:
    if predictions_vgg16[i][0] > predictions_vgg16[i][1]:
        print("Negative")
    else:
        print("Positive")  

VGG16 Predictions: 
Negative
Negative
Positive
Positive
Negative


In [27]:
print("ResNet Predictions: ")
for i in indices:
    if predictions_resnet[i][0] > predictions_resnet[i][1]:
        print("Negative")
    else:
        print("Positive")

ResNet Predictions: 
Negative
Negative
Positive
Positive
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).